# Показатели блоков
Пример расчета показателей блоков с группировкой по типу распространения

## Описание задачи и условий расчета
- Период: 01.09.2023 - 30.09.2023
- ЦА: Все 4+, Все 18+, Все 14-44, Все 10-45
- Место просмотра: Все места (Дом+Дача)
- Каналы: НТВ, СТС, ТНТ
- Блоки
- Статистики: CumReach(000), Rtg% (тотал), Quantity (тотал)

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# В справочнике демографических атрибутов найдем нужные нам переменные
cats.get_tv_demo_attribute(names=['возраст'])

# Для формирования условий запроса нам понадобятся значения столбца entityName: age

In [ ]:
#В справочнике телекомпаний найдем нужные нам каналы и выведем их id
cats.get_tv_company(name=["НТВ (СЕТЕВОЕ ВЕЩАНИЕ)", 
                          "СТС (СЕТЕВОЕ ВЕЩАНИЕ)", 
                          "ТНТ (СЕТЕВОЕ ВЕЩАНИЕ)"])['id'].tolist()

## Формирование задания
Зададим условия расчета

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. Можно указать несколько периодов
date_filter = [('2023-09-01', '2023-09-30')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter=None

# Задаем каналы: НТВ, СТС, ТНТ
company_filter = 'tvCompanyId IN (1862, 1867, 1870)'

# Указываем фильтр программ
program_filter = None

# Фильтр блоков
break_filter = None

# Фильтр роликов: статус - реальный
ad_filter = None

# Указываем список срезов (группировок)
slices = [
    'researchMonth', #месяц
    'tvCompanyName', # телекомпания
    'breaksDistributionTypeName' #Блок распространение
]

# Указываем список статистик для расчета
statistics = ['CumReach000', 'RtgPerSum', 'QuantitySum']

# Задаем условия сортировки: месяц (от новых к старым), телекомпания (от а до я), Блок распространение (от а до я)
sortings = {'researchMonth':'ASC','tvCompanyName':'ASC','breaksDistributionTypeName':'ASC'}

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all
    "issueType": "BREAKS" #Тип события - Блоки
}

Сформируем словарь с целевыми аудиториями

In [ ]:
# Задаем необходимые группы
targets = {
    'a. Все 4+':'age >= 4',
    'b. Все 18+':'age >= 18',
    'c. Все 14-44':'age >= 14 and age <= 44',
    'd. Все 10-45':'age >= 10 and age <= 45'
}

## Расчет задания

In [ ]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой ЦА формируем задание и отправляем на расчет
for target, syntax in targets.items():
    
    # Подставляем значения словаря в параметры
    project_name = target 
    basedemo_filter = syntax
      
    # Формируем задание для API TV Index в формате JSON
    task_json = mtask.build_crosstab_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                        daytype_filter=daytype_filter, company_filter=company_filter, 
                                        location_filter=location_filter, basedemo_filter=basedemo_filter, 
                                        targetdemo_filter=targetdemo_filter,program_filter=program_filter, 
                                        break_filter=break_filter, ad_filter=ad_filter, 
                                        slices=slices, statistics=statistics, sortings=sortings, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_crosstab_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')
    
print(f"\nid: {[i['task']['taskId'] for i in tasks]}") 

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)

# Приводим порядок столбцов в соответствие с условиями расчета
df = df[['prj_name']+slices+statistics]

df

## Настройка внешнего вида таблицы
Пропустите этот шаг, если хотите экспортировать таблицу в ее текущем виде

In [ ]:
# Формируем сводную таблицу: строки - срезы, столбцы - ЦА, значения - статистики
df = pd.pivot_table(df, values=statistics,
                        index=slices, 
                        columns=['prj_name'])
df

In [ ]:
# Опционально: поменять местами уровни в столбцах
df = df.swaplevel(0,1, axis=1).sort_index(axis=1)

df

## Сохраняем в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('09_breaks')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)